# Code 1 - 49조

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import requests
from bs4 import BeautifulSoup
import time
import openpyxl
import pandas as pd

### 엑셀 파일 생성하기
파일명, 시트명, 컬럼명을 정해줍니다.

In [2]:
#크롤링 데이터 저장할 리스트 생성
ws = []         
ws.append(['브랜드','상품명','카테고리','정가','할인가','아이디','별점','피부정보','피부타입','피부고민','자극도'])  #컬럼명 제공
ws

[['브랜드',
  '상품명',
  '카테고리',
  '정가',
  '할인가',
  '아이디',
  '별점',
  '피부정보',
  '피부타입',
  '피부고민',
  '자극도']]

In [3]:
######################제공하는 코드 건드리지 마세요 #############################

#크롬 드라이버 자동 업데이트
from webdriver_manager.chrome import ChromeDriverManager

#브라우저 꺼짐 방지
chrome_options = Options()
chrome_options.add_experimental_option('detach',True)

#불필요한 에러 메세지 없애기
chrome_options.add_experimental_option('excludeSwitches', ['enable-logging'])

service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

### 올리브영 스킨케어 랭킹 링크를 main_url로 두고 10위까지 제품 상세 링크를 sub_url에 저장하기

In [4]:
main_url = 'https://www.oliveyoung.co.kr/store/main/getBestList.do?dispCatNo=900000100100001&fltDispCatNo=10000010001&pageIdx=1&rowsPerPage=8'

response = requests.get(main_url)
html = response.text
soup = BeautifulSoup(html, 'html.parser')

# CSS 선택자를 수정하여 올바른 요소를 대상으로 선택합니다.
links = soup.select("#Container > div.best-area > div.TabsConts.on > ul > li > div > a")

sub_url = []

# enumerate 함수를 사용하여 rank를 자동으로 처리
for rank, link in enumerate(links, 1):
    sub_url.append(link['href'])
    if rank >= 10:
        break

print(sub_url)

['https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000187781&dispCatNo=90000010009&trackingCd=Best_Sellingbest', 'https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000166360&dispCatNo=90000010009&trackingCd=Best_Sellingbest', 'https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000188240&dispCatNo=90000010009&trackingCd=Best_Sellingbest', 'https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000188189&dispCatNo=90000010009&trackingCd=Best_Sellingbest', 'https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000156230&dispCatNo=90000010009&trackingCd=Best_Sellingbest', 'https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000166675&dispCatNo=90000010009&trackingCd=Best_Sellingbest', 'https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000180464&dispCatNo=90000010009&trackingCd=Best_Sellingbest', 'https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?g

### 크롤링 함수 만들기
제품 상세 페이지에서 상단 제품정보(브랜드명,상품명,카테고리,정가,할인가)와  
하단의 리뷰고객 좌픅정보(아이디,별점,피부정보)와 우측정보(피부타입,피부고민,자극도)를 가져옵니다.  
위의 데이터를 엑셀에 계속 추가하는 함수를 만듭니다. 

In [5]:
def customer_info():
    
    soup = BeautifulSoup(driver.page_source, "html.parser")

    for j in range(0,10,1):

        #브랜드
        try:
            brand = driver.find_element(By.ID, 'onlBrndNm').get_attribute('value')
        except:
            brand ="없음"

        #상품명
        try:
            p_name = driver.find_element(By.ID, 'goodsNm').get_attribute('value')
        except:
            p_name="없음"

        #카테고리
        try:
            p_category = driver.find_element(By.ID, 'eigeneSmlDispName').get_attribute('value')
        except:
            p_category="없음"

        #정가
        try:
            price = soup.select_one("#Contents > div.prd_detail_box.renew > div.right_area > div > div.price > span.price-1 > strike").text.strip()
        except:
            price=0

        #할인가
        try:            
            discount = soup.select_one('#Contents > div.prd_detail_box.renew > div.right_area > div > div.price > span.price-2 > strong').text.strip() 
        except:
            discount=0    
            
        #고객 아이디
        try:
            _id=soup.select_one(f'#gdasList > li:nth-child({j+1}) > div.info > div > p.info_user > a.id').text.strip()
        except:
            _id ="없음"        


        #별점
        try:
            _star = soup.select_one(f'#gdasList > li:nth-child({j+1}) > div.review_cont > div.score_area > span.review_point > span').text.strip() 
        except:
            _star="없음"

        #고객 피부 정보
        try:
            _info = soup.select_one(f'#gdasList > li:nth-child({j+1}) > div.info > div > p.tag').text.strip() 
        except:
            _info = "없음"


        #피부 타입
        try:
            skin_type = soup.select_one(f'#gdasList > li:nth-child({j+1}) > div.review_cont > div.poll_sample > dl:nth-child(1) > dd > span').text.strip()
        except:
            skin_type = "없음"
            
        #피부 고민
        try:
            skin_trouble = soup.select_one(f'#gdasList > li:nth-child({j+1}) > div.review_cont > div.poll_sample > dl:nth-child(2) > dd > span').text.strip()
        except:
            skin_trouble = "없음"
        #자극도
        try:
            skin_irritation = soup.select_one(f'#gdasList > li:nth-child({j+1}) > div.review_cont > div.poll_sample > dl:nth-child(3) > dd > span').text.strip()
        except:
            skin_irritation = "없음"
            
        #엑셀 데이터 쌓기
        ws.append([brand,p_name,p_category,price,discount,_id, _star, _info, skin_type, skin_trouble, skin_irritation])
        time.sleep(1)

### 웹페이지 해당 주소로 이동하여 크롤링함수를 실행하는 반복문을 만듭니다.
반복문에 들어가야 하는 것  
1. 리뷰버튼 클릭  
2. 화면 맨 아래까지 스크롤하기 (코드제공)
3. 10페이지 이하일 때 : 10페이지 크롤링 한 후 다음페이지 화살표 버튼 누르기 , 마지막 페이지인 경우 last_page==True
4. 11페이지부터 규칙을찾아 다음페이지로 이동하게 만들기, 마지막 페이지인 경우 last_page==True

In [6]:
#웹페이지 해당 주소 이동
for i in range(0,10):          #전체 제품을 한번에 크롤링하지 않고 나눠서 크롤링 할 경우 sub_url 인덱스 고려해서 range숫자 변경

    driver = webdriver.Chrome()  # 웹 드라이버 재시작
    driver.implicitly_wait(5)  #웹페이지 로딩 될때까지 5초는 기다림
    driver.maximize_window()   #화면 최대화
    driver.get(sub_url[i])          
    time.sleep(3)


    #리뷰버튼 클릭
    rv = driver.find_element(By.XPATH, '//*[@id="reviewInfo"]/a') #셀레니움 최신버전을 사용하는 관계로 해당 코드로 수정했습니다
    #rv = driver.find_element_by_xpath('//*[@id="reviewInfo"]/a')
    rv.click()
    time.sleep(3)
    

    #리뷰 하단 끝까지 스크롤하는 함수 (빈칸없음.그대로 코드 사용가능)
    before_h = driver.execute_script("return window.scrollY")         #스크롤 전 높이
    
    #화면 맨아래까지 스크롤
    while True:
        driver.find_element(By.CSS_SELECTOR,"body").send_keys(Keys.END)

        time.sleep(1)

        #스크롤 후 높이
        after_h = driver.execute_script("return window.scrollY")

        #스크롤 값이 같으면 스크롤 멈춤
        if after_h == before_h:
            break
        before_h = after_h           
        
        
    last_page=False
    
    for k in range(1,101):  #100페이지 크롤링 한다고 했을 때 (상품당 최대 100페이지까지 있음)

        #마지막 페이지면 종료
        if last_page == True:
            break
        
        #10페이지 이하일 때와 11페이지 이상일 때 각각 코드를 작성하지 않고 k의 변화에 따라 move를 변화시켜 한번에 처리함
        #11페이지 이상인 경우 move는 2~11이 반복됨
        if k<11:
            move=k
        else:
            move=(k%10)+1
            if move==1:
                move=11

        try:
            customer_info()
            pg = driver.find_element(By.XPATH, f'//*[@id="gdasContentsArea"]/div/div[8]/a[{move}]').click()      
            #pg = driver.find_element(By.XPATH, f'//*[@id="gdasContentsArea"]/div/div[8]/a[{move}]').click() #일부 제품은 페이지 넘기기가 dif[7]에 있음
            

        except:
            last_page = True
                   

driver.quit()

### 크롤링한 결과를 엑셀에 저장 (상단에서 만든 엑셀 파일명과 동일하게)

In [7]:
columns = ['브랜드', '상품명', '카테고리', '정가', '할인가', '아이디', '별점', '피부정보', '피부타입', '피부고민', '자극도']
df = pd.DataFrame(ws[1:], columns=columns)
df.head()

,브랜드,상품명,카테고리,정가,할인가,아이디,별점,피부정보,피부타입,피부고민,자극도
0,구달,[홍현희 PICK/ 한정기획] 구달 청귤 비타C 잡티케어 세럼 50ml+50ml 리필기획,에센스/크림,"43,000","28,900",고먐미먼지,5점만점에 3점,복합성 겨울쿨톤 미백 블랙헤드,지성에 좋아요,주름/미백에 좋아요,자극없이 순해요
1,구달,[홍현희 PICK/ 한정기획] 구달 청귤 비타C 잡티케어 세럼 50ml+50ml 리필기획,에센스/크림,"43,000","28,900",뚜딘,5점만점에 5점,복합성 웜톤 다크서클 모공,복합성에 좋아요,주름/미백에 좋아요,자극없이 순해요
2,구달,[홍현희 PICK/ 한정기획] 구달 청귤 비타C 잡티케어 세럼 50ml+50ml 리필기획,에센스/크림,"43,000","28,900",수부지지만행복해묘,5점만점에 5점,지성 겨울쿨톤 모공 미백,건성에 좋아요,주름/미백에 좋아요,자극없이 순해요
3,구달,[홍현희 PICK/ 한정기획] 구달 청귤 비타C 잡티케어 세럼 50ml+50ml 리필기획,에센스/크림,"43,000","28,900",뽀드득뽀드득,5점만점에 5점,민감성 여름쿨톤 민감성 잡티,복합성에 좋아요,주름/미백에 좋아요,보통이에요
4,구달,[홍현희 PICK/ 한정기획] 구달 청귤 비타C 잡티케어 세럼 50ml+50ml 리필기획,에센스/크림,"43,000","28,900",요니양,5점만점에 5점,복합성 여름쿨톤 각질 모공,복합성에 좋아요,진정에 좋아요,보통이에요


In [8]:
df.to_excel('Top Rank 10.xlsx', index=False)